# Analysis of a dataset using data mining techniques.

This notebook focuses on exploring and mining basic information from an anonymised retail transactions dataset given by the company Instacart.

## Taking a first glance at the dataset
*Source code for this section may be find in file `dist/first-glance.class.ts`* 

The dataset consists of information about 3.4 million grocery orders, distributed across 6 `.csv` files listed below:

In [1]:
import * as cp from 'child_process';

/**
 * Function executes a child_process listing the files in the instacart_basket_data folder
 * and returns the listed files though a Promise.
 */
function listFiles(): Promise<string[]> {
    // Async console for Jupyter
    $$async$$ = true;
    // Async behavior
    return new Promise( (resolve) => {
        // Listing the files in the instacart_basket_data folder.
        cp.exec(`ls instacart_basket_data`)
            .stdout.on('data', (data: string) => {
                // Formatting ls output as an Array of strings (representing the file names)
                resolve(data.match(/[^\r\n]+/g));
            });
    });
}

listFiles().then( (files: string[]) => {
    console.log(files);
    $$done$$();
});

[ 'aisles.csv',
  'departments.csv',
  'order_products__train.csv',
  'orders.csv',
  'products.csv' ]


Files composing the dataset are listed below:

```
[ 'aisles.csv',
  'departments.csv',
  'order_products__prior.csv',
  'order_products__train.csv',
  'orders.csv',
  'products.csv' ]
```

As a starting point, in order to have a first glance of the data we will actually be playing with throughout this entire report; let us display the first items composing each `.csv` file listed above. 

We will use NodeJS's straight-forward File I/O fs to list, open and `'csv-parse'`'s `Parser` to parse `.csv` files.

In [2]:
import * as fs from 'fs';
import { Parser } from 'csv-parse';

/**
 * Function reads a .csv file and returns it properlly formatted.
 */
function readFile<T>(filePath: string): Promise<{ data:Array<T>, file: string }> {
    // Async console for Jupyter
    $$async$$ = true;
    // Async behavior
    return new Promise( (resolve, reject) => {
        let ret: Array<T> = [];

        // 'csv-parse' Parser, columns options groups each row by column in an object
        let parser: Parser = new Parser({
                delimiter: ',',
                columns: true
        });

        parser
            .on('data', (data: T) => ret.push(data) )
            .on('error', (error: any) => reject(error) )
            .on('end', () => resolve({data: ret, file: filePath}) );

        // Reading the file and piping to parser
        fs.createReadStream(filePath).pipe(parser);
    });
}

listFiles().then( (files: string[]) => {
    console.log('Parsing all the files, this may take some time... (I mean, a REALLY long time)');
    // Creating promises array to run readFile() of files concurrently
    let promises: Promise<any>[] = files.map( (file: string) => {
        // Formatting to get absolute paths to files.
        let filePath: string = `instacart_basket_data/${file}`;
        // Reading file
        return readFile<any>(filePath);
    });
    // Concurrent execution of promises
    return Promise.all(promises);
}).then( (results: Array<any[]>) => {
    // Reading first two elements of each file
    results.forEach( (result: Array<any>) => {
        console.log(`First two lines of file: ${result.file}`); 
        console.log(result.data.slice(0,2)); 
    });
    // Async console for Jupyter
    $$done$$();
}).catch( (e) => console.log(e) );

Parsing all the files, this may take some time... (I mean, a REALLY long time)
First two lines of file: instacart_basket_data/aisles.csv
[ { aisle_id: '1', aisle: 'prepared soups salads' },
  { aisle_id: '2', aisle: 'specialty cheeses' } ]
First two lines of file: instacart_basket_data/departments.csv
[ { department_id: '1', department: 'frozen' },
  { department_id: '2', department: 'other' } ]
First two lines of file: instacart_basket_data/order_products__train.csv
[ { order_id: '1',
    product_id: '49302',
    add_to_cart_order: '1',
    reordered: '1' },
  { order_id: '1',
    product_id: '11109',
    add_to_cart_order: '2',
    reordered: '1' } ]
First two lines of file: instacart_basket_data/orders.csv
[ { order_id: '2539329',
    user_id: '1',
    eval_set: 'prior',
    order_number: '1',
    order_dow: '2',
    order_hour_of_day: '08',
    days_since_prior_order: '' },
  { order_id: '2398795',
    user_id: '1',
    eval_set: 'prior',
    order_number: '2',
    order_dow: '3',


File `aisles.csv` is structured as such; and contains the two following rows:

| aisle_id | aisle |
|----------|-------|
| 1 | prepared soups salads |
| 1 | specialty cheeses |

File `departments.csv` is structured as such; and contains the two following rows:

| department_id | department |
|---------------|------------|
| 1 | frozen |
| 2 | other |


File `products.csv` is structured as such; and contains the two following rows:

| product_id | product_name | aisle_id | department_id | 
|------------|--------------|----------|---------------|
| 1 | Chocolate Sandwich Cookies | 61 | 19 |
| 2 | All-Seasons Salt | 104 | 13 |


Files `aisles.csv`, `departments.csv`, `products.csv` and all (trivially) contain information about the aisles, products, and departments names respectivelly; which may not be of any interet to us other than for enhanced visualisation. We'll thus have to perform the proper `JOIN`s (a.k.a. `UNION`) between these tables and our future data / pattern collections when needed.

File `orders.csv` is surely a bit more interresting, especially having in mind **sequential pattern mining**, as it lists all the orders, and contains information on **when** and **by who** it has been placed.

Finally, files `order_products__train.csv` and `order_products__prior.csv` contain the same, and most valuable information in regard to pattern mining, as they contain products ordered within each order.

## Warming up : First statistics over the dataset
*Source code for this section may be find in file `dist/stats.class.ts`* 

We'll keep our first analysis of the data simple, and start by computing some simple statistics over the dataset, on the number of orders,  products, products per order, etc. 
In addition to understanding more about the data, this will also allow us to find some appropriate and revelant criteria on which we could base (and reduce) our dataset for extended itemset analysis; as well as giving us a first glance at pattern dedundancy. 

Keeping in mind that each row of the `order_products__prior.csv` file has the following structure:
- order_id
- product_id
- add_to_cart_order
- reordered

An interesting approach would be to regroup these objects by both `order_id` and `product_id`, as it would allow us to have a glance over the product distribution through the orders on the first hand, as well as idea of each product's popularity on the other.

To do so, we need to transform the dataset consequently. Thus, let's start by defining `ProductOrder` as the structure of the data outputted by the `.csv` file parser:

From this point, as we focus on exploiting data from file `order_products__prior.csv`, which contains more than 1 million records; and for the sake of memory usage, we'll try to work on data streams as much as possible, rather than parsing 1-million-elements-cached `Arrays` when it comes to data transformation.
We'll thus be using Reactive Programming library `RxJS` in that intent.

Reactive programming is nothing new as it only consists in programming with asynchronous data streams, which languages like JS are basically all about. `RxJS` yet provides us with an amazing and complete approach -as well as a great toolbox of functions- to combine, create and filter such streams easily.

### Grouping by orders
*Sources are available in file `dist/stats-on-orders.spaghetti.ts`.*

Let us define a function allowing us to group `ProductOrder` objects by any key of the `ProductOrder` interface. We'll use in that intent `RxJS`'s `groupBy` method, which basically groups the items emitted by an `Observable` (a.k.a. stream; in our case, the `ReadStream` of the considered file) according to a specified criterion (in our case, either the `product_id` or the `order_id`), and emits these grouped items as `GroupedObservable`s, one `GroupedObservable` per group.

Let's start by defining a `Group<T>` as an object containg an `id` (the grouping criterion basically), as well as an `Array` of whatever item of type `T` we're grouping. This will be one of many product of our function:

We'll also forge ahead (keeping pattern mining in mind) by allowing this method to `.map()` the `ProductOrder` objects to whatever we want (its `id`, `product_name`...) depending on our need.

Such a function is given below:

In [3]:
import { Observable } from 'rxjs/Observable';
import 'rxjs/add/operator/finally';
import 'rxjs/add/operator/groupBy';
import * as RxNode from 'rx-node';

/**
 * Function returns an Observable of `ProductOrder` group by a defined criterion. You may map the parsed `ProductOrder` to whatever value you want.
 */
function _readAndGroupBy<T>( key: keyof ProductOrder, map: (val: ProductOrder) => T ): Rx.Observable<Group<T>> {
    /**
     * 'csv-parse' Parser, columns options groups each row by column in an object.
     */
    let parser: Parser = new Parser({
        delimiter: ',',
        columns: true
    });

    // Turning native stream into Observable
    return RxNode.fromStream( fs.createReadStream(`instacart_basket_data/order_products__train.csv`).pipe(parser) )
        // Grouping objects by order
        .groupBy( (data: ProductOrder) => data[key] )
        // At this point, we basically have an Observable by group. Thus we need to flatten that.
        .flatMap( (group: Rx.GroupedObservable<string, ProductOrder>) => {
            return group
                // Formatting the data
                .map(map)
                // And flattening the Observable array.
                .reduce( (concat: Group<T>, current: T) => {
                    concat.items.push(current);
                    return concat;
                }, {
                    id: group.key,
                    items: []
                })
        });
}

{}

Let us group the `ProductOrder` by their `order_id`. `ProductOrders` will be caracterized by their `product_id` (We'll thus trivially have an list of Orders (or itemsets), as `Arrays` of `product_id`s). 

Function above will return us with all the processed groups. We'll compute some basic statistics on these from there, such as: 
- The number of orders (number of groups);
- The minimum number of product in an order (minimum of arrays length);
- The maximum number of product in an order (maximum of arrays length);
- The average product number per order (average array length);
- The number of records in the `order_products__prior.csv` file (sum of arrays length);

In [4]:
function statsOnOrders(): void {
    $$async$$ = true;
    console.log('Gathering data, this might take a while...');
    
    /**
     * All the groups.
     */
    let groups: Array<Group<string>> = [];

    let stats: any = {
        max: 0,
        min: Infinity,
        sum: 0
    }

    /**
     * Reads the file and groups `ProductOrders as intended`
     */

    _readAndGroupBy<string>('order_id', (productOrder: ProductOrder) => productOrder.product_id )
        // Once all groups are loaded, displaying them.
        .finally( () => {
            console.log(`Maximum number of ProductOrders: ${stats.max}`);
            console.log(`Minimum number of ProductOrders: ${stats.min}`);
            console.log(`Average number of ProductOrders: ${stats.sum / groups.length}`);
            console.log(`Total number of ProductOrders: ${stats.sum}`);
            console.log(`Number of itemsets: ${groups.length}`);
            $$done$$();
        })
        // Note that this behaviour (induced by the flatMap of readAndGroupBy) makes everything pretty much blocking again.
        .subscribe( (group: Group<string>) => {
            // Computing some basic stats on the fly
            stats.max = Math.max(group.items.length, stats.max);
            stats.min = Math.min(group.items.length, stats.min);
            stats.sum += group.items.length;

            // Pushing group to groups.
            groups.push(group)
        });
}

statsOnOrders();

Gathering data, this might take a while...
Maximum number of ProductOrders: 80
Minimum number of ProductOrders: 1
Average number of ProductOrders: 10.552759338155157
Total number of ProductOrders: 1384617
Number of itemsets: 131209


Grouping the records by their `order_id` enlights us of the following information:

| Number of orders | Minimum product number per order | Maximum product number per order | Average product number per order | Total number of records |
|-|-|-|-|-|-|
| 131,209 | 1 | 80 | 10.6 | 1,384,617 |

Some trivial modifications of the function above could allow us to retrieve the number of product per order for enhanced visualisation (code can be find in source files): 

### Grouping by product

Creating itemsets (`Groups`) of Orders, based on the `product_id` of `ProductOrders` may also be of interest to us, as it allow us "feel" a product "popularity" by counting the number of orders it appears in. This is a pretty good deal in regards to pattern mining, as:
- a "frequent" product will be more likely to appear in frequent itemsets;
- its number of appearance in the collection is, by definition, the maximum support over the dataset. Considering a Product A being the most popular in a dataset such as ours, the itemset `{ A }` will trivially be the absolute, most frequent itemset to be find in the entire dataset;
- if a product is too frequent, it may be of interest to ignore it, as the itemsets to be find may not be revealant enough.

Code is basically the same as before, thus won't be included in the notebook. Sources are however still available in file `dist/stats-on-products.spaghetti.ts`.
Grouping the records by their `product_id` gives us the following results:


| Number of products | Minimum order number per product | Maximum order number per product | Average order number per product | Total number of records |
|-|-|-|-|-|-|
| 391,23 | 1 | 18,726 | 35.39 | 1,384,617 |

Joining retrieved data with table `products.csv` using the (dirty, yet it works.) following function: 

In [5]:
/**
 * Function finds element of array of which the key corresponds to value; and returns another defined value of this element.
 */
function join<T>( array: T[], initKey: keyof T, value: any, returnKey: keyof T): any {
    let element: T = array.find((element: T) => element[initKey] == value );
    return element[returnKey];
}

undefined

... we are able to compare products popularity, and conclude on: 
- **Banana** being the most popular product, being ordered 18,726 times;
- **46 products** including 100% Black Cherry & Concord Grape Juice, Breaded Popcorn Turkey Dogs or Lip Balm, are the least popular with only 1 order.

## Frequent item sets
Knowing a little bit more about our data, we'll now move to mine and gather **frequent** itemsets from our training dataset (`order_products_train.csv`). In other words, we will focus on finding regularities in the shopping behavior of customers, based on what `product` orders (a.k.a. transactions) are composed of.

### Vocabulary
Let us define some basic vocabulary we will be tremendously be using:
- Let I be a set of items. An itemset is a subset of I.
- Let D be a transaction database such that each transaction is an itemset.
- Frequency of an itemset is the number of transactions including the itemset.
- For a given support, an itemset is said to be frequent if its frequency is no less than the support.

### SPMF Open-Source Library
Many (if not all) algorithms tested upon this point will be implementations from SPMF library (http://www.philippe-fournier-viger.com/spmf/); an open-source data mining mining library written in Java, specialized in pattern mining.

A NodeJS JS/TypeScript wrapper, `class SPMF` for SPMF (spawning Java commands) has been written in order to execute these, while still exploiting the results with Typescript. Sources may be find in file `./src/class/spmf.ts`. Examples of its use are shown below.


### Apriori
Keeping our goal in mind, we'll start by using the **Apriori Algorithm**. The object of this algorithm is to identify association between different sets of data, and to find out patterns in data using a "bottom up approach", as frequent subsets are  extended one item at a time; and is based on a property called the anti-monotone property: 
It states that if an item set is not frequent, then none of its supersets can be frequent. As a result, the list of  potential frequent itemset gets smaller as mining progresses [1].

Though the Apriori Algorithm is easy to implement and understand, it is far from being performant the reasons stated above.

#### Dataset formatting
Upon this point we will be using SPMF library's Java implementation of Apriori (through command lines), in order to mine frequent item sets from our dataset. This implementation needs the input data to be formatted as such: 

- An item is represented by a positive integer.
- A transaction is a line in the file.
- In each line (transaction), items are separated by a single space.
- It is assumed that all items within a same transaction (line) are sorted according to a total order and that no item can appear twice within the same line.

For example, an input file is defined as follows:

```
1 2 5
2 7
```

Transforming this dataset into this format is pretty straightforward using the code we already wrote previously (same code, but encapsulated in a `CSVParser` class). To save us some time, already formatted data will be available to parse (file `output/formatted-itemsets.csv`). Here is an example of `CSVParser` though, running on a custom 6-line dataset:

In [6]:
// Our custom parser class, looks like Jupyter/iTypescript doesn't like ES6 imports of custom Typescript classes.
import * as CSVParser from './dist/class/csv-parser.class.js';
import { ProductOrder } from './src/interface/product-order.interface';
import { Group } from './src/interface/group.interface';

function formatData(): void {
    $$async$$ = true;
    console.log('Generating transactions, this might take a while...');
    
    let groups: Group<ProductOrder,number>[] = [];

    new CSVParser.CSVParser<ProductOrder>(`output/small_order_products__train.csv`)
        // Grouping items by order_id, and mapping every item composing these itemsets to their product_id.
        .generateItemsets<number>('order_id', (productOrder: ProductOrder) => Number.parseInt(productOrder.product_id))
        // Once execution is complete, writing the formatted dataset into a proper file.
        .finally( () => {
            // Writing number of product per order_id in a new file : The array of already formatted rows is joined by a return carriage character.
            console.log('Example of output:');
            console.log(groups.join('\r\n'));
            $$done$$();
        })
        // On group reception, formatting the items composing is as a ROW (joined by plain space character), and pushing it the output array.
        .subscribe((group: Group<ProductOrder,number>) => groups.push(group.items.sort( (a: number, b: number) => a - b ).join(' ')) )
}

formatData();


Generating transactions, this might take a while...
Example of output:
1
101 1002
10 100 2000


#### Parsing the results
This Apriori implementation enlights us with multiple information other than the frequent itemsets themselves, as it outputs multiple results through both `stdout`, including:
- The number of candidates;
- The maximum size of candidates the algorithm stopped at;
- The frequent itemsets count;
- Maximum memory usage;
- Total execution time.

Mined frequent itemsets may be find in the specified output file, formatted as below:

```
...A #SUP: B

-- EXAMPLES OF OUTPUT --
49628 #SUP: 186
49683 #SUP: 2413
13176 21137  #SUP: 164
13176 21903  #SUP: 175
```

Where `...A` represents the spread of `item_ids` composing the frequent itemset; and `B` the support of this itemset (in term of number of occurency).

Eventhough the output format is not rigorously `.csv` friendly, we can still easily parse the output file with our TypeScript application with no addtionnal dependency or need of code; using a little hack, defining the string ` #SUP: ` as a `.csv` separator. This will result in us parsing an Array of 2-item rows:

```
[ ...item_ids, support ]
```
This is own our homemade SPMF wrapper inherently works.

Last step will trivially be to parse those item_ids and joining these results with the `products.csv` table for enhanced visualisation.


#### Maximum support

In the extend of running the Apriori Algorithm, we need to define a proper minimum support value, as defined above. This is a crucial step, as setting the support to an exagerated value would result no itemsets to be find; while setting it to a too low value will inherently result in a very long execution time.

As seen previously, `Banana` is the most frequent item in out dataset, being recorded in 18,726 of the 131,209 transactions i.e. 14.3% of all the orders.
This number also represents the maximum support of any itemset in our dataset, as being the support of itemset composed of the `Banana` singleton. 

#### First runs

Let us run Apriori a few times, with a different minimum support threshold, in order to compare the execution time as well as the number of return candidates. One small drawback of this implementation being that it returns single-item candidates (which are not of interest at all), we'll also specify the number on multiple-items candidates found.

In [7]:
import * as SPMF from './dist/class/spmf.class.js';
import { SPMFResults, ItemSet, Rule } from './class/spmf.class';

function aprioriRun(): void {
    $$async$$ = true;
    console.log('Mining patterns, this might take a while...');
    // Our custom SPMF wrapper
    new SPMF.SPMF('Apriori')
        // Loading from file
        .fromFile(`output/formatted_itemsets.csv`)
        // Executes Apriori with 5% support
        .exec<ItemSet>(5)
        // Listening for results
        .subscribe((results: SPMFResults<ItemSet>) => {
            // Wrapper returns both the stats...
            console.log('Stats:');
            console.log(results.stats);
            // ... and the frequent itemsets. Showing the first two itemsets:
            console.log('Itemsets:');
            console.log(results.output);

            $$done$$();
        });
}

aprioriRun();

Mining patterns, this might take a while...
Stats:
{ candidates: 28, executionTime: 535, memory: 61.69075012207031 }
Itemsets:
[ { support: 15480, items: [ '13176' ] },
  { support: 10894, items: [ '21137' ] },
  { support: 9784, items: [ '21903' ] },
  { support: 18726, items: [ '24852' ] },
  { support: 7293, items: [ '47209' ] },
  { support: 8135, items: [ '47626' ] },
  { support: 7409, items: [ '47766' ] } ]


| Minimum support | Number of candidates | Frequent itemsets count | Multiple-items itemsets | Execution time |
|-----------------|----------------------|-------------------------|-------------------------|----------------|
| 13%| 1 (Banana, as expected) | 1    | 0  | 422 ms |
| 7% | 10                      | 4    | 0  | 423 ms |
| 3% | 153                     | 17   | 0  | 748 ms |
| 1% | 5,467                   | 120  | 14 | 8.2 s  |
|0.5%| 33,042                  | 364  | 108| 48.9 s |
|0.3%| 210,892                 | 1125 | 478| 4 mn 58 s |

***TODO***

As expected, the speed of Apriori gets in the way pretty fast when it comes to parsing a dataset of such size, and with a maximum support so low. In order to find more relevant itemsets, we may have to transform the dataset in order reduce the overall total number of candidates.

#### Our first frequent itemsets

***TODO***


### LCM Algorithm

LCM is an algorithm know to be the fastest for mining frequent **closed** itemsets; knowing that an itemset X is closed in a dataset S if no proper super-itemset Y that has the same support count as X in S exists.

We will be using SPMF library's JAVA implementation of LCM, in order to mine frequent closed itemsets from our dataset;
An advantage beingi that this implementation inputs the same file format as the Apriori Algorithm of the same library, thus, there is no need to format our dataset in a different manner.

#### First runs

It would be interesting to compare the performances and results of the LCM and Apriori algorithms; thus, let's run LCM over the same dataset again, and with the same support. You can also try it by yourself by executing the code below:

In [10]:
import * as SPMF from './dist/class/spmf.class.js';

function lcmRun(): void {
    $$async$$ = true;
    console.log('Mining patterns, this might take a while...');
    // Our custom SPMF wrapper
    new SPMF.SPMF('LCM')
        // Loading from file
        .fromFile(`output/formatted_itemsets.csv`)
        // Executes LCM with 5% support
        .exec<ItemSet>(5)
        // Listening for results
        .subscribe((results: SPMFResults<ItemSet>) => {
            // Wrapper returns both the stats...
            console.log('Stats:');
            console.log(results.stats);
            // ... and the frequent itemsets. Showing the first two itemsets:
            console.log('Itemsets:');
            console.log(results.output);

            $$done$$();
        });
}

$$done$$();
lcmRun();

Mining patterns, this might take a while...
Stats:
{ candidates: undefined, executionTime: 307, memory: undefined }
Itemsets:
[ { support: 15480, items: [ '13176' ] },
  { support: 10894, items: [ '21137' ] },
  { support: 9784, items: [ '21903' ] },
  { support: 18726, items: [ '24852' ] },
  { support: 7293, items: [ '47209' ] },
  { support: 8135, items: [ '47626' ] },
  { support: 7409, items: [ '47766' ] } ]


| Minimum support | Frequent itemsets count | Multiple-items itemsets | Execution time |
|-----------------|----------------------|-------------------------|-------------------------|----------------|
| 13%| 1    | 0  | 204 ms |
| 7% | 4    | 0  | 275 ms |
| 3% | 17   | 0  | 328 ms |
| 1% | 120  | 16 | 1.2 s  |
|0.5%| 364  | 108| 2.3 s  |
|0.3%| 844  | 339| 4.9 s  |

### Quick comparison 

***TODO***


## Frequent item sets association rules

Once patterns have been mined, we can also determine association rules, as each item may have several relations with others. These relations thus indirect relationships between the items composing the transactions; expressed with a certain **confidence**, as an example:

`Diapers implies beer with 65% confidence`

It is obvious valuable way to format thr data for direct marketing, sales promotions, and for discovering business trends.

### FPGrowth association mining

Association rules mining generally consist in two steps: the first step being to discover frequent itemsets (With Apriori, LCM or any other algorithm as we did previously); and the second, to generate rules by using these frequent itemsets.

We'll be using in that order yet another all-in-one Algorithm from the SPMF Library, the `FPGrowth_association_rules` algorithm, which basically mine item sets using a faster, more memory efficient algorithm than Apriori: `FPGrowth`; before generating the association rules.

Algorithm takes 2 input values:
- The minimum support of mined itemsets, as before;
- The minimum confidence of resulting association rules;

#### Parsing the results

Parsing the results may defer a bit, as the output file now indicates rules instead of plain itemsets, the support of the association, as well as the confidence of the rule.

Example of output:
```
1 ==> 2 4 5 #SUP: 3 #CONF: 0.75
5 6 ==> 1 2 4 #SUP: 3 #CONF: 0.6
4 7 ==> 1 #SUP: 3 #CONF: 0.75
```

Such a file would result in being parsed the following way with our current code:

```
['1 ==> 2 4 5', '3 #CONF: 0.75']
```

Nothing to worry about though, as a really hacky to do it would be to `split()` both resulting strings using separators ` ==> ` and ` #CONF `. I'd really wish I would have use Regular expressions at this point though...

#### Late update: Regular expressions

That was I ended up doing eventually, keeping in mind that we will need to parse outpit files from even more algorithms (sequential pattern mining, etc...). I'm thus using following Regular expressions:
```
Matching itemsets:
/^(\d )+#SUP: \d*\t*\n*\r*$\g

Matching association rules:
/^(\d )+==> (\d )+#SUP: \d #CONF: \d+.\d*\t*\n*\r*$/g
```

To match results as they are parsed from output files; and using even more regular expressions within the matches of the previous ones to parse for numeric values.


***TODO***

#### Running the algorithm

You can try the algorithm by yourself, by editing and executing the code below:

In [11]:
import * as SPMF from './dist/class/spmf.class.js';

function associationRulesMining(): void {
    $$async$$ = true;
    console.log('Mining association rules, this might take a while...');
    // Our custom SPMF wrapper
    new SPMF.SPMF('FPGrowth_association_rules')
        // Loading from file
        .fromFile(`output/formatted_itemsets.csv`)
        // Executes LCM with 0.1% support and 25% confidence
        .exec<Rule>(0.1,25)
        // Listening for results
        .subscribe((results: SPMFResults<Rule>) => {
            // Wrapper returns both the stats...
            console.log('Stats:');
            console.log(results.stats);
            // ... and the frequent rules. Showing the first two rules:
            console.log('Rules:');     
            console.log(results.output);

            $$done$$();
        });
}

$$done$$();
associationRulesMining();

Mining association rules, this might take a while...
{ support: 198,
  confidence: 0,
  items: [ '45' ],
  results: [ '24852' ] }
{ support: 136,
  confidence: 0,
  items: [ '46149' ],
  results: [ '196' ] }
{ support: 311,
  confidence: 0,
  items: [ '260' ],
  results: [ '13176' ] }
{ support: 330,
  confidence: 0,
  items: [ '260' ],
  results: [ '21137' ] }
{ support: 331,
  confidence: 0,
  items: [ '260' ],
  results: [ '24852' ] }
{ support: 170,
  confidence: 0,
  items: [ '329' ],
  results: [ '13176' ] }
{ support: 378,
  confidence: 0,
  items: [ '432' ],
  results: [ '24852' ] }
{ support: 256,
  confidence: 0,
  items: [ '1158' ],
  results: [ '24852' ] }
{ support: 152,
  confidence: 0,
  items: [ '1244' ],
  results: [ '13176' ] }
{ support: 232,
  confidence: 0,
  items: [ '1463' ],
  results: [ '13176' ] }
{ support: 145,
  confidence: 0,
  items: [ '1511' ],
  results: [ '24852' ] }
{ support: 396,
  confidence: 0,
  items: [ '2295' ],
  results: [ '15290' ] }
{ suppo

{ support: 169,
  confidence: 0,
  items: [ '13263' ],
  results: [ '28465' ] }
{ support: 157,
  confidence: 0,
  items: [ '13517' ],
  results: [ '24852' ] }
{ support: 197,
  confidence: 0,
  items: [ '13629' ],
  results: [ '21137' ] }
{ support: 147,
  confidence: 0,
  items: [ '13733' ],
  results: [ '24852' ] }
{ support: 157,
  confidence: 0,
  items: [ '14218' ],
  results: [ '24852' ] }
{ support: 154,
  confidence: 0,
  items: [ '14897' ],
  results: [ '24852' ] }
{ support: 331,
  confidence: 0,
  items: [ '14992' ],
  results: [ '24852' ] }
{ support: 487,
  confidence: 0,
  items: [ '15290' ],
  results: [ '24852' ] }
{ support: 221,
  confidence: 0,
  items: [ '15937' ],
  results: [ '47626' ] }
{ support: 288,
  confidence: 0,
  items: [ '21288' ],
  results: [ '16797' ] }
{ support: 1948,
  confidence: 0,
  items: [ '16797' ],
  results: [ '24852' ] }
{ support: 201,
  confidence: 0,
  items: [ '17600' ],
  results: [ '21137' ] }
{ support: 197,
  confidence: 0,
  item

{ support: 150,
  confidence: 0,
  items: [ '4605', '16797' ],
  results: [ '24852' ] }
{ support: 188,
  confidence: 0,
  items: [ '4605', '21903' ],
  results: [ '24852' ] }
{ support: 148,
  confidence: 0,
  items: [ '4605', '21903' ],
  results: [ '47626' ] }
{ support: 138,
  confidence: 0,
  items: [ '4605', '47766' ],
  results: [ '21903' ] }
{ support: 138,
  confidence: 0,
  items: [ '4605', '21903' ],
  results: [ '47766' ] }
{ support: 153,
  confidence: 0,
  items: [ '4605', '26209' ],
  results: [ '24852' ] }
{ support: 135,
  confidence: 0,
  items: [ '4605', '45007' ],
  results: [ '24852' ] }
{ support: 257,
  confidence: 0,
  items: [ '4605', '47626' ],
  results: [ '24852' ] }
{ support: 190,
  confidence: 0,
  items: [ '4605', '47766' ],
  results: [ '24852' ] }
{ support: 179,
  confidence: 0,
  items: [ '4605', '26209' ],
  results: [ '47626' ] }
{ support: 157,
  confidence: 0,
  items: [ '4605', '47766' ],
  results: [ '47626' ] }
{ support: 235,
  confidence: 0,

{ support: 141,
  confidence: 0,
  items: [ '13176', '16759' ],
  results: [ '47209' ] }
{ support: 143,
  confidence: 0,
  items: [ '18465', '21137' ],
  results: [ '13176' ] }
{ support: 143,
  confidence: 0,
  items: [ '13176', '18465' ],
  results: [ '21137' ] }
{ support: 282,
  confidence: 0,
  items: [ '19057', '21137' ],
  results: [ '13176' ] }
{ support: 282,
  confidence: 0,
  items: [ '13176', '19057' ],
  results: [ '21137' ] }
{ support: 196,
  confidence: 0,
  items: [ '19057', '21903' ],
  results: [ '13176' ] }
{ support: 200,
  confidence: 0,
  items: [ '19057', '27966' ],
  results: [ '13176' ] }
{ support: 240,
  confidence: 0,
  items: [ '19057', '47209' ],
  results: [ '13176' ] }
{ support: 587,
  confidence: 0,
  items: [ '21137', '21903' ],
  results: [ '13176' ] }
{ support: 587,
  confidence: 0,
  items: [ '13176', '21903' ],
  results: [ '21137' ] }
{ support: 202,
  confidence: 0,
  items: [ '21137', '22035' ],
  results: [ '13176' ] }
{ support: 202,
  con

{ support: 497,
  confidence: 0,
  items: [ '21903', '47209' ],
  results: [ '13176' ] }
{ support: 269,
  confidence: 0,
  items: [ '13176', '47766' ],
  results: [ '21903' ] }
{ support: 166,
  confidence: 0,
  items: [ '21903', '48679' ],
  results: [ '13176' ] }
{ support: 166,
  confidence: 0,
  items: [ '13176', '48679' ],
  results: [ '21903' ] }
{ support: 152,
  confidence: 0,
  items: [ '22035', '47209' ],
  results: [ '13176' ] }
{ support: 152,
  confidence: 0,
  items: [ '13176', '22035' ],
  results: [ '47209' ] }
{ support: 182,
  confidence: 0,
  items: [ '22825', '47209' ],
  results: [ '13176' ] }
{ support: 182,
  confidence: 0,
  items: [ '13176', '22825' ],
  results: [ '47209' ] }
{ support: 268,
  confidence: 0,
  items: [ '22935', '24964' ],
  results: [ '13176' ] }
{ support: 268,
  confidence: 0,
  items: [ '13176', '24964' ],
  results: [ '22935' ] }
{ support: 268,
  confidence: 0,
  items: [ '13176', '22935' ],
  results: [ '24964' ] }
{ support: 135,
  con

{ support: 160,
  confidence: 0,
  items: [ '21137', '24964' ],
  results: [ '22935' ] }
{ support: 142,
  confidence: 0,
  items: [ '22935', '27966' ],
  results: [ '21137' ] }
{ support: 174,
  confidence: 0,
  items: [ '21137', '22935' ],
  results: [ '47209' ] }
{ support: 221,
  confidence: 0,
  items: [ '21137', '26209' ],
  results: [ '24852' ] }
{ support: 315,
  confidence: 0,
  items: [ '24852', '27966' ],
  results: [ '21137' ] }
{ support: 147,
  confidence: 0,
  items: [ '21137', '28204' ],
  results: [ '24852' ] }
{ support: 246,
  confidence: 0,
  items: [ '24852', '39275' ],
  results: [ '21137' ] }
{ support: 134,
  confidence: 0,
  items: [ '21137', '43961' ],
  results: [ '24852' ] }
{ support: 199,
  confidence: 0,
  items: [ '21137', '45066' ],
  results: [ '24852' ] }
{ support: 331,
  confidence: 0,
  items: [ '21137', '47626' ],
  results: [ '24852' ] }
{ support: 379,
  confidence: 0,
  items: [ '21137', '47766' ],
  results: [ '24852' ] }
{ support: 167,
  con

{ support: 184,
  confidence: 0,
  items: [ '22935', '47209' ],
  results: [ '24964' ] }
{ support: 145,
  confidence: 0,
  items: [ '22935', '47626' ],
  results: [ '24964' ] }
{ support: 151,
  confidence: 0,
  items: [ '22935', '31717' ],
  results: [ '26209' ] }
{ support: 151,
  confidence: 0,
  items: [ '22935', '26209' ],
  results: [ '31717' ] }
{ support: 141,
  confidence: 0,
  items: [ '22935', '47626' ],
  results: [ '26209' ] }
{ support: 141,
  confidence: 0,
  items: [ '22935', '26209' ],
  results: [ '47626' ] }
{ support: 132,
  confidence: 0,
  items: [ '22935', '27966' ],
  results: [ '47209' ] }
{ support: 143,
  confidence: 0,
  items: [ '24184', '47626' ],
  results: [ '24852' ] }
{ support: 188,
  confidence: 0,
  items: [ '24184', '47766' ],
  results: [ '24852' ] }
{ support: 208,
  confidence: 0,
  items: [ '24964', '47626' ],
  results: [ '24852' ] }
{ support: 208,
  confidence: 0,
  items: [ '24852', '24964' ],
  results: [ '47626' ] }
{ support: 197,
  con

{ support: 140,
  confidence: 0,
  items: [ '13176', '21137', '30391' ],
  results: [ '47209' ] }
{ support: 134,
  confidence: 0,
  items: [ '24852', '47626', '47766' ],
  results: [ '21903' ] }
{ support: 134,
  confidence: 0,
  items: [ '21903', '47626', '47766' ],
  results: [ '24852' ] }
{ support: 134,
  confidence: 0,
  items: [ '21903', '24852', '47766' ],
  results: [ '47626' ] }
{ support: 134,
  confidence: 0,
  items: [ '21903', '24852', '47626' ],
  results: [ '47766' ] }
{ support: 146,
  confidence: 0,
  items: [ '26209', '47626', '47766' ],
  results: [ '24852' ] }
{ support: 146,
  confidence: 0,
  items: [ '24852', '47626', '47766' ],
  results: [ '26209' ] }
{ support: 146,
  confidence: 0,
  items: [ '24852', '26209', '47766' ],
  results: [ '47626' ] }
{ support: 146,
  confidence: 0,
  items: [ '24852', '26209', '47626' ],
  results: [ '47766' ] }
Stats:
{ candidates: undefined, executionTime: 35, memory: undefined }
Rules:
[ { support: 198,
    confidence: 0,
   

## Sequential patterns


## References

[1]: "Kumar, Dr K Ramesh & D, Usha. (2013). Frequent Pattern Mining Algorithm for Crime Dataset: An Analysis. INTERNATIONAL JOURNAL OF ENGINEERING SCIENCES & RESEARCH.".
Avalaible on https://www.researchgate.net/publication/266498986_Frequent_Pattern_Mining_Algorithm_for_Crime_Dataset_An_Analysis

